In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [3]:
!wget $data -O data-week-3.csv 

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
df = pd.read_csv('data.csv')

In [5]:
df.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [6]:
df.columns = df.columns.str.replace(' ', '_').str.lower()

In [7]:
df.head()

,make,model,year,engine_fuel_type,engine_hp,engine_cylinders,transmission_type,driven_wheels,number_of_doors,market_category,vehicle_size,vehicle_style,highway_mpg,city_mpg,popularity,msrp
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [10]:
features_to_use = ['make','model','year','engine_hp', 'engine_cylinders', 'transmission_type', 'vehicle_style', 
                   'highway_mpg', 'city_mpg', 'msrp']

In [12]:
df = df[features_to_use]

In [16]:
df = df.rename(columns = {'msrp':'price'})

In [17]:
df.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500


In [18]:
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

In [19]:
categorical_columns

['make', 'model', 'transmission_type', 'vehicle_style']

In [20]:
for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [21]:
df.head().T

,0,1,2,3,4
make,bmw,bmw,bmw,bmw,bmw
model,1_series_m,1_series,1_series,1_series,1_series
year,2011,2011,2011,2011,2011
engine_hp,335.0,300.0,300.0,230.0,230.0
engine_cylinders,6.0,6.0,6.0,6.0,6.0
transmission_type,manual,manual,manual,manual,manual
vehicle_style,coupe,convertible,coupe,coupe,convertible
highway_mpg,26,28,28,28,28
city_mpg,19,19,20,18,18
price,46135,40650,36350,29450,34500


In [22]:
df.isnull().sum()

make                  0
model                 0
year                  0
engine_hp            69
engine_cylinders     30
transmission_type     0
vehicle_style         0
highway_mpg           0
city_mpg              0
price                 0
dtype: int64

In [23]:
df.engine_hp = df.engine_hp.fillna(0)
df.engine_cylinders = df.engine_cylinders.fillna(0)

In [24]:
df.isnull().sum()

make                 0
model                0
year                 0
engine_hp            0
engine_cylinders     0
transmission_type    0
vehicle_style        0
highway_mpg          0
city_mpg             0
price                0
dtype: int64

In [33]:
df.transmission_type.value_counts()

automatic           8266
manual              2935
automated_manual     626
direct_drive          68
unknown               19
Name: transmission_type, dtype: int64

In [111]:
numerical = ['year', 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg']

In [41]:
df[numerical].corrwith(df.price).abs().sort_values(ascending=False)

engine_hp           0.650095
engine_cylinders    0.526274
year                0.227590
highway_mpg         0.160043
city_mpg            0.157676
dtype: float64

In [42]:
avg_price = df.price.mean()

In [43]:
avg_price

40594.737032063116

In [68]:
df['above_average'] = df.price > avg_price

In [69]:
df.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price,above_average
0,bmw,1_series_m,2011,335.0,6.0,manual,coupe,26,19,46135,True
1,bmw,1_series,2011,300.0,6.0,manual,convertible,28,19,40650,True
2,bmw,1_series,2011,300.0,6.0,manual,coupe,28,20,36350,False
3,bmw,1_series,2011,230.0,6.0,manual,coupe,28,18,29450,False
4,bmw,1_series,2011,230.0,6.0,manual,convertible,28,18,34500,False


In [70]:
df.above_average = df.above_average.astype(int)

In [71]:
df.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price,above_average
0,bmw,1_series_m,2011,335.0,6.0,manual,coupe,26,19,46135,1
1,bmw,1_series,2011,300.0,6.0,manual,convertible,28,19,40650,1
2,bmw,1_series,2011,300.0,6.0,manual,coupe,28,20,36350,0
3,bmw,1_series,2011,230.0,6.0,manual,coupe,28,18,29450,0
4,bmw,1_series,2011,230.0,6.0,manual,convertible,28,18,34500,0


In [72]:
from sklearn.model_selection import train_test_split

In [73]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [74]:
len(df_train), len(df_val), len(df_test)

(7148, 2383, 2383)

In [76]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [77]:
y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values

del df_train['above_average']
del df_val['above_average']
del df_test['above_average']

In [78]:
from sklearn.metrics import mutual_info_score

In [79]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.above_average)

In [81]:
mi = round(df_full_train[categorical_columns].apply(mutual_info_churn_score),2)
mi.sort_values(ascending=False)

model                0.46
make                 0.24
vehicle_style        0.08
transmission_type    0.02
dtype: float64

In [127]:
from sklearn.feature_extraction import DictVectorizer

In [128]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical_columns + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical_columns + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [129]:
dv.get_feature_names()

['city_mpg',
 'engine_cylinders',
 'engine_hp',
 'highway_mpg',
 'make=acura',
 'make=alfa_romeo',
 'make=aston_martin',
 'make=audi',
 'make=bentley',
 'make=bmw',
 'make=bugatti',
 'make=buick',
 'make=cadillac',
 'make=chevrolet',
 'make=chrysler',
 'make=dodge',
 'make=ferrari',
 'make=fiat',
 'make=ford',
 'make=genesis',
 'make=gmc',
 'make=honda',
 'make=hummer',
 'make=hyundai',
 'make=infiniti',
 'make=kia',
 'make=lamborghini',
 'make=land_rover',
 'make=lexus',
 'make=lincoln',
 'make=lotus',
 'make=maserati',
 'make=maybach',
 'make=mazda',
 'make=mclaren',
 'make=mercedes-benz',
 'make=mitsubishi',
 'make=nissan',
 'make=oldsmobile',
 'make=plymouth',
 'make=pontiac',
 'make=porsche',
 'make=rolls-royce',
 'make=saab',
 'make=scion',
 'make=spyker',
 'make=subaru',
 'make=suzuki',
 'make=tesla',
 'make=toyota',
 'make=volkswagen',
 'make=volvo',
 'model=100',
 'model=124_spider',
 'model=190-class',
 'model=1_series',
 'model=2',
 'model=200',
 'model=200sx',
 'model=240',

In [130]:
from sklearn.linear_model import LogisticRegression

In [131]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)

In [132]:
model.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [133]:
model.intercept_[0]

-0.8094486080229117

In [134]:
model.coef_[0].round(3)

array([ 8.900e-02, -2.700e-02,  3.700e-02, -7.000e-03,  1.554e+00,
        2.456e+00,  9.240e-01,  3.165e+00,  2.520e-01,  3.032e+00,
        0.000e+00, -5.730e-01,  2.825e+00, -1.737e+00, -1.505e+00,
       -4.195e+00,  6.610e-01, -8.540e-01, -2.271e+00,  1.193e+00,
       -1.500e+00, -1.594e+00, -4.980e-01, -2.659e+00,  6.270e-01,
       -1.653e+00,  2.000e-02,  2.078e+00,  1.574e+00,  1.441e+00,
        4.572e+00,  1.643e+00,  1.000e-02, -2.131e+00,  1.000e-03,
        1.060e+00, -2.741e+00, -1.213e+00, -2.022e+00, -1.230e+00,
       -4.011e+00,  2.075e+00,  1.565e+00,  9.880e-01, -3.780e-01,
        6.010e-01, -3.382e+00, -2.448e+00,  3.834e+00, -6.250e-01,
       -9.640e-01,  1.222e+00, -1.162e+00, -1.050e-01, -2.200e-02,
       -2.203e+00, -2.000e-03, -1.574e+00, -4.000e-03, -1.100e-02,
       -5.000e-03, -2.766e+00, -2.210e-01,  8.880e-01, -8.870e-01,
       -6.600e-02, -3.180e-01, -4.730e-01, -0.000e+00, -3.500e-02,
       -1.140e+00,  2.610e-01,  2.280e-01,  6.100e-02,  2.945e

In [135]:
y_pred = model.predict_proba(X_val)[:, 1]

In [136]:
y_pred

array([3.98505754e-04, 9.97204477e-01, 3.94201081e-05, ...,
       2.39115229e-04, 9.95904892e-01, 9.93326890e-01])

In [137]:
above_avg_decision = (y_pred >= 0.5)

In [138]:
(y_val == above_avg_decision).mean()

0.9471254720939991

In [139]:
# excluding year
dv = DictVectorizer(sparse=False)
exclude_year = ['make', 'model', 'transmission_type', 'vehicle_style', 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg']

train_dict = df_train[exclude_year].to_dict(orient='records')

X_train = dv.fit_transform(train_dict)

val_dict = df_val[exclude_year].to_dict(orient='records')

X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict_proba(X_val)[:, 1]

above_avg_decision = (y_pred >= 0.5)

(y_val == above_avg_decision).mean()

0.9479647503147294

In [140]:
# excluding engine_hp
dv = DictVectorizer(sparse=False)
exclude_year = ['make', 'model', 'transmission_type', 'vehicle_style', 'year', 'engine_cylinders', 'highway_mpg', 'city_mpg']

train_dict = df_train[exclude_year].to_dict(orient='records')

X_train = dv.fit_transform(train_dict)

val_dict = df_val[exclude_year].to_dict(orient='records')

X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict_proba(X_val)[:, 1]

above_avg_decision = (y_pred >= 0.5)

(y_val == above_avg_decision).mean()

0.9307595467897608

In [141]:
# excluding transmission_type
dv = DictVectorizer(sparse=False)
exclude_year = ['make', 'model', 'engine_hp', 'vehicle_style', 'year', 'engine_cylinders', 'highway_mpg', 'city_mpg']

train_dict = df_train[exclude_year].to_dict(orient='records')

X_train = dv.fit_transform(train_dict)

val_dict = df_val[exclude_year].to_dict(orient='records')

X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict_proba(X_val)[:, 1]

above_avg_decision = (y_pred >= 0.5)

(y_val == above_avg_decision).mean()

0.9450272765421738

In [142]:
# excluding city_mpg
dv = DictVectorizer(sparse=False)
exclude_year = ['make', 'model', 'engine_hp', 'vehicle_style', 'year', 'engine_cylinders', 'highway_mpg', 'transmission_type']

train_dict = df_train[exclude_year].to_dict(orient='records')

X_train = dv.fit_transform(train_dict)

val_dict = df_val[exclude_year].to_dict(orient='records')

X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict_proba(X_val)[:, 1]

above_avg_decision = (y_pred >= 0.5)

(y_val == above_avg_decision).mean()

0.9324381032312211

In [147]:
year_diff = 0.9471254720939991 - 0.9479647503147294
engine_hp_diff = 0.9471254720939991  - 0.9307595467897608
transmission_type_diff = 0.9471254720939991 - 0.9450272765421738
city_mpg_diff = 0.9471254720939991 - 0.9324381032312211

In [158]:
df_differences = pd.DataFrame({"year_diff":[year_diff],
                        "engine_hp_diff":[engine_hp_diff],
                        "transmission_type_diff":[transmission_type_diff],
                        "city_mpg_diff":[city_mpg_diff]})

In [159]:
df_differences

,year_diff,engine_hp_diff,transmission_type_diff,city_mpg_diff
0,-0.000839,0.016366,0.002098,0.014687


In [161]:
df_train.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,mitsubishi,endeavor,2011,225.0,6.0,automatic,4dr_suv,19,15,33599
1,kia,borrego,2009,276.0,6.0,automatic,4dr_suv,21,17,26245
2,lamborghini,gallardo,2012,570.0,10.0,manual,convertible,20,12,248000
3,chevrolet,colorado,2016,200.0,4.0,automatic,crew_cab_pickup,27,20,24990
4,pontiac,vibe,2009,158.0,4.0,automatic,4dr_hatchback,26,20,20475


In [164]:
df_train_linear = df_train.copy()
df_val_linear = df_val.copy()
df_test_linear = df_test.copy()

In [165]:
df_train_linear.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,mitsubishi,endeavor,2011,225.0,6.0,automatic,4dr_suv,19,15,33599
1,kia,borrego,2009,276.0,6.0,automatic,4dr_suv,21,17,26245
2,lamborghini,gallardo,2012,570.0,10.0,manual,convertible,20,12,248000
3,chevrolet,colorado,2016,200.0,4.0,automatic,crew_cab_pickup,27,20,24990
4,pontiac,vibe,2009,158.0,4.0,automatic,4dr_hatchback,26,20,20475


In [166]:
y_train_orig = df_train_linear.price.values
y_val_orig = df_val_linear.price.values
y_test_orig = df_test_linear.price.values

In [167]:
y_train_linear = np.log1p(y_train_orig)
y_val_linear = np.log1p(y_val_orig)
y_test_linear = np.log1p(y_test_orig)

del df_train_linear['price']
del df_val_linear['price']
del df_test_linear['price']

In [168]:
df_train_linear.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg
0,mitsubishi,endeavor,2011,225.0,6.0,automatic,4dr_suv,19,15
1,kia,borrego,2009,276.0,6.0,automatic,4dr_suv,21,17
2,lamborghini,gallardo,2012,570.0,10.0,manual,convertible,20,12
3,chevrolet,colorado,2016,200.0,4.0,automatic,crew_cab_pickup,27,20
4,pontiac,vibe,2009,158.0,4.0,automatic,4dr_hatchback,26,20


In [169]:
dv = DictVectorizer(sparse=False)

train_dict_linear = df_train_linear.to_dict(orient='records')
X_train_linear = dv.fit_transform(train_dict_linear)

val_dict_linear = df_val_linear.to_dict(orient='records')
X_val_linear = dv.transform(val_dict_linear)

In [170]:
dv.get_feature_names()

['city_mpg',
 'engine_cylinders',
 'engine_hp',
 'highway_mpg',
 'make=acura',
 'make=alfa_romeo',
 'make=aston_martin',
 'make=audi',
 'make=bentley',
 'make=bmw',
 'make=bugatti',
 'make=buick',
 'make=cadillac',
 'make=chevrolet',
 'make=chrysler',
 'make=dodge',
 'make=ferrari',
 'make=fiat',
 'make=ford',
 'make=genesis',
 'make=gmc',
 'make=honda',
 'make=hummer',
 'make=hyundai',
 'make=infiniti',
 'make=kia',
 'make=lamborghini',
 'make=land_rover',
 'make=lexus',
 'make=lincoln',
 'make=lotus',
 'make=maserati',
 'make=maybach',
 'make=mazda',
 'make=mclaren',
 'make=mercedes-benz',
 'make=mitsubishi',
 'make=nissan',
 'make=oldsmobile',
 'make=plymouth',
 'make=pontiac',
 'make=porsche',
 'make=rolls-royce',
 'make=saab',
 'make=scion',
 'make=spyker',
 'make=subaru',
 'make=suzuki',
 'make=tesla',
 'make=toyota',
 'make=volkswagen',
 'make=volvo',
 'model=100',
 'model=124_spider',
 'model=190-class',
 'model=1_series',
 'model=2',
 'model=200',
 'model=200sx',
 'model=240',

In [171]:
from sklearn.linear_model import Ridge

In [174]:
Ridge?

In [175]:
model_linear = Ridge(solver='sag', alpha = 0, random_state=42)

In [176]:
model_linear.fit(X_train_linear, y_train_linear)

Ridge(alpha=0, random_state=42, solver='sag')

In [185]:
y_pred_linear = model_linear.predict(X_val_linear)

In [186]:
y_pred_linear

array([10.23776114, 11.00608674,  9.20406634, ...,  9.05405293,
       11.50145677, 10.84589318])

In [188]:
def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

In [189]:
rmse(y_val_linear, y_pred_linear)

0.494009829930089

In [190]:
rmses = []

for a in [0, 0.01, 0.1, 1, 10]:
    model_linear = Ridge(solver='sag', alpha = a, random_state=42)
    model_linear.fit(X_train_linear, y_train_linear)
    y_pred_linear = model_linear.predict(X_val_linear)
    result = rmse(y_val_linear, y_pred_linear)
    
    print(a, result)
    
    rmses.append(result)

0 0.494009829930089
0.01 0.4940099838768778
0.1 0.49401136935655604
1 0.4940252207146743
10 0.49425040974751566
